In [7]:
#Librerias utilizadas
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import datetime
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt
nltk.download("punk")
nltk.download("stopwords")

[nltk_data] Error loading punk: Package 'punk' not found in index
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importando conjunto de datos de la plataforma HIPGive para entrenar algoritmo de sentimientos

In [8]:
# Datos generados por mi 
#data_hip_comments=pd.read_csv('data/hip_give_message_data_for_training_sentimental_analysis.csv')

# Datos proporcionados de hip_hive
data_hipGive=pd.read_excel('data/HIPGive_Análisis_de_comentarios.xlsx',sheet_name='Sheet1',encode='utf-8')

In [9]:
# Limpienado datos en comentarios y categorías
#features 
#labels 

message_data_hipGive = data_hipGive[[u'Message', u'Positive/Negative/Neutral']]
message_data_hipGive = message_data_hipGive.dropna()

features_hipGive = message_data_hipGive['Message']
labels_hipGive   = message_data_hipGive['Positive/Negative/Neutral']

In [10]:
# Transformación a lista
features_train=features_hipGive.tolist()
labels_train=labels_hipGive.tolist()

In [11]:
# Limpieza de "labels_train"
# Es importante limpiar los labels_train de imperfecciones en su escritura 
# para no tener dos textos diferentes para la misma clasificación
labels_cleaned=[]
for sentence in range(0, len(labels_train)):
    # Remove all the special characters    
    processed_feature = re.sub(r'\W', ' ', str(labels_train[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    
    #Removiendo espacios en blanco
    processed_feature =  processed_feature.strip()
    
    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    labels_cleaned.append(processed_feature)
labels_train=labels_cleaned

In [12]:
# Limpieza de "features_train"
# Es importante limpiar los features_train de imperfecciones en su escritura 
# para entrenar bien a el algorítmo
processed_features = []

for sentence in range(0, len(features_train)):
    # Remove all the special characters
    #print(features_train[sentence])
    processed_feature = re.sub(r'\W', ' ', str(features_train[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

## Representando los comentarios en forma numérica para poder entrenar el algorítmo

In [13]:
# Técnica "TF-IDF"
# processed_features: regresa un array de vectores con elementos 0 y 1. Cada vector representa un comentario.
vectorizer = TfidfVectorizer (max_features=2000, min_df=7, max_df=0.8, stop_words=stopwords.words('english')+stopwords.words('spanish'))
processed_features = vectorizer.fit_transform(processed_features).toarray()

In [14]:
processed_features

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Dividiendo datos en un conjunto para entrenar y uno para realizar pruebas
### Todos los datos se utilizaran para entrenamiento, por tanto, test_size=0. 

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels_train, test_size=0.0, random_state=0)

## Entrenando el modelo

In [17]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

# Aplicando el modelo a los comentarios de donantes

In [18]:
# Lectura de base de datos donde se encuentran los comentarios de donantes
data = pd.read_excel('../STANFORD_ HIPGive_By_country_2014_2019_JUNUETH.xlsx',sheet_name='All funders data 2014-2019')
message=data['Message']

In [19]:
# Limpieza de valores nulos en la columna de comentarios
message_crean=message.dropna()

In [20]:
np_message_crean= message_crean.values # COmentarios de donantes dentro de una estructura numpy.array

In [21]:
# Limpieza de "np_message_crean"
# Es importante limpiar los np_message_crean (comentarios) 
# de imperfecciones en su escritura 
# para aplicar bien dentro del algorítmo
cleaned_features_message = []

for sentence in range(0, len(np_message_crean)):
    # Remove all the special characters
    cleaned_message = re.sub(r'\W', ' ', np_message_crean[sentence])

    # remove all single characters
    cleaned_message= re.sub(r'\s+[a-zA-Z]\s+', ' ', cleaned_message)

    # Remove single characters from the start
    cleaned_message = re.sub(r'\^[a-zA-Z]\s+', ' ', cleaned_message) 

    # Substituting multiple spaces with single space
    cleaned_message = re.sub(r'\s+', ' ', cleaned_message, flags=re.I)

    # Removing prefixed 'b'
    cleaned_message = re.sub(r'^b\s+', '', cleaned_message)

    # Converting to Lowercase
    cleaned_message = cleaned_message.lower()

    cleaned_features_message.append(cleaned_message)

In [22]:
len(cleaned_features_message)

5325

### Vectorización de comentarios 

In [23]:
# Obtención de palabras que conforman la "Caja de palabra"
bag_of_words = vectorizer.get_feature_names()

In [24]:
# Transformando comentarios de donantes a texto tokenized
# dentro de la matrix matrix_of_tokenized_comments
# para después vectorizarlo a entradas 0 y 1.
matrix_of_tokenized_comments=[]
for coment in cleaned_features_message:
    tokenized_comment=nltk.word_tokenize(coment)    
    matrix_of_tokenized_comments.append(tokenized_comment)

In [25]:
len(matrix_of_tokenized_comments)

5325

In [26]:
# Palabras sin significado
useless_words=nltk.corpus.stopwords.words("spanish")+nltk.corpus.stopwords.words("english")+list(string.punctuation)+[u'cada',u'través',u'para',u'\xa1que',u'\xa1to',u'son',u'con',u'la',u'se',u'en',u'el',u'que',u'la',u'las',u'lo',u'los',u'de',u'the',u'us',u'we',u'A',u'nbsp',u'8217',u'8211',u'8220',u'8221',u'\u2019',u'\u201d',u'\u201c',u'pa\xeds',u'ray\xf3n',u'pa\xeds',u'ay\xfadanos',u'\xa1que',u'harder\u2014but',u'let\xb4s',u'\xa1to',u'granny\xb4s',u'espa\xf1ol.',u'.',u'nmilc'] #+list(u"\u2019")+list('...')

In [27]:
# Limpiando los comentarios dentro de matrix_of_tokenized_comments
# de palabras sin significado
matrix_of_cleaned_tokenized_comments=[]
for tokenized_comment in matrix_of_tokenized_comments:
    comment_tokenized_cleaned=[]
    for word in tokenized_comment:
        #print (word)
        if not word in useless_words:            
            comment_tokenized_cleaned.append(word)        
    matrix_of_cleaned_tokenized_comments.append(comment_tokenized_cleaned)    

In [28]:
# Vectorizando comentarios dentro de matrix_of_cleaned_tokenized_comments a 0's y 1's.
matrix_of_vectorized_comments=[]
for cleaned_tokenized_comment in matrix_of_cleaned_tokenized_comments:
    vector_comment=[]
    for word in bag_of_words:
        if word in cleaned_tokenized_comment:
            vector_comment.append(1)
        else:
            vector_comment.append(0)
    matrix_of_vectorized_comments.append(vector_comment)

In [29]:
matrix_of_vectorized_comments[0].count(1)+matrix_of_vectorized_comments[0].count(0)

70

In [30]:
len(matrix_of_vectorized_comments)

5325

## Predicción sobre comentarios

In [31]:
predictions_of_comments = text_classifier.predict(matrix_of_vectorized_comments) #numpy.array

In [32]:
len(predictions_of_comments)

5325

In [33]:
set(predictions_of_comments)

{'neutral', 'positive'}

In [34]:
predictions_of_comments.tolist().count('positive'),predictions_of_comments.tolist().count('negative'),predictions_of_comments.tolist().count('neutral')

(5260, 0, 65)

In [35]:
total=float(predictions_of_comments.tolist().count('positive')+predictions_of_comments.tolist().count('negative')+predictions_of_comments.tolist().count('neutral'))
(100*predictions_of_comments.tolist().count('negative')/total,100*predictions_of_comments.tolist().count('neutral')/total,100*predictions_of_comments.tolist().count('positive')/total)

(0.0, 1.2206572769953052, 98.77934272300469)

# Analizando los comentarios 

In [36]:
# Creación de función que obtiene las palabras
# más usadas en un conjunto de comentarios
def get_most_common_words_in_comments(lts):
    all_coments=[]
    for comment in lts:
        all_coments.extend(comment)
        
    words_counter=Counter(all_coments)
    most_common_words = words_counter.most_common()
    return most_common_words

In [37]:
neg_index_lst=[]
neg_i=0
neu_index_lst=[]
neu_i=0
pos_index_lst=[]
pos_i=0

for item in predictions_of_comments:
    neg_i=neg_i+1
    neu_i=neu_i+1
    pos_i=pos_i+1
    if item=='negative':        
        neg_index_lst.append(neg_i)
    elif item=='neutral':        
        neu_index_lst.append(neu_i)
    elif item=='positive':        
        pos_index_lst.append(pos_i)        

In [38]:
neg_comments=[]
neu_comments=[]
pos_comments=[]

for neg_index in neg_index_lst:
    neg_comments.append(matrix_of_cleaned_tokenized_comments[neg_index-1])
for neu_index in neu_index_lst:
    neu_comments.append(matrix_of_cleaned_tokenized_comments[neu_index-1])
for pos_index in pos_index_lst:
    pos_comments.append(matrix_of_cleaned_tokenized_comments[pos_index-1])
    

In [39]:
neu_words=get_most_common_words_in_comments(neu_comments)
neg_words=get_most_common_words_in_comments(neg_comments)
pos_words=get_most_common_words_in_comments(pos_comments)

In [40]:
neu_words

[(u'maria', 25),
 (u'donacion', 22),
 (u'mochila', 18),
 (u'rico', 3),
 (u'puerto', 3),
 (u'years', 2),
 (u'suffering', 2),
 (u'gloria', 2),
 (u'plaza', 2),
 (u'fondo', 2),
 (u'liz', 2),
 (u'mujeres', 2),
 (u'irma', 2),
 (u'donation', 2),
 (u'mejia', 1),
 (u'hurricanes', 1),
 (u'dance', 1),
 (u'hurricane', 1),
 (u'embassy', 1),
 (u'kate', 1),
 (u'osg', 1),
 (u'mejores', 1),
 (u'computaci', 1),
 (u'vida', 1),
 (u'pride', 1),
 (u'deseo', 1),
 (u'good', 1),
 (u'alumnae', 1),
 (u'amo', 1),
 (u'creciendo', 1),
 (u'embajada', 1),
 (u'marcy', 1),
 (u'elena', 1),
 (u'association', 1),
 (u'conejo', 1),
 (u'fernanda', 1),
 (u'soys', 1),
 (u'duran', 1),
 (u'lillian', 1),
 (u'nivia', 1),
 (u'sager', 1),
 (u'necesitan', 1),
 (u'small', 1),
 (u'amor', 1),
 (u'quintero', 1),
 (u'candi', 1),
 (u'magdalena', 1),
 (u'siga', 1),
 (u'redes', 1),
 (u'j', 1),
 (u'people', 1),
 (u'sembrando', 1),
 (u'esfuerzo', 1),
 (u'past', 1),
 (u'urruela', 1),
 (u'chavez', 1),
 (u'leslie', 1),
 (u'erika', 1),
 (u'peter',

In [41]:
pos_words

[(u'work', 657),
 (u'keep', 359),
 (u'thank', 354),
 (u'good', 327),
 (u'great', 318),
 (u'gracias', 235),
 (u'love', 223),
 (u'support', 210),
 (u'project', 196),
 (u'adelante', 181),
 (u'proyecto', 180),
 (u'felicidades', 168),
 (u'luck', 137),
 (u'trabajo', 137),
 (u'go', 135),
 (u'help', 134),
 (u'behalf', 130),
 (u'thanks', 130),
 (u'community', 122),
 (u'mujeres', 119),
 (u'amazing', 113),
 (u'suerte', 113),
 (u'puerto', 110),
 (u'important', 110),
 (u'rico', 109),
 (u'labor', 108),
 (u'honor', 108),
 (u'happy', 107),
 (u'gran', 104),
 (u'proud', 102),
 (u'wonderful', 100),
 (u'hope', 98),
 (u'sigan', 92),
 (u'cause', 90),
 (u'donation', 88),
 (u'best', 87),
 (u'much', 76),
 (u'si', 74),
 (u'children', 74),
 (u'dios', 73),
 (u'puede', 73),
 (u'xito', 71),
 (u'women', 71),
 (u'people', 68),
 (u'please', 65),
 (u'give', 63),
 (u'muchas', 63),
 (u'program', 62),
 (u'excelente', 61),
 (u'causa', 59),
 (u'apoyo', 59),
 (u'need', 57),
 (u'sli', 56),
 (u'mejor', 55),
 (u'organization', 

### Procesando texto para worldCloud

In [42]:
def text_for_Wordcloud(lis):
    text=u''
    for element in lis:
        text=text+u' '+(element[0]+u' ')*element[1]
    return text

In [43]:
neg_worldCloud=text_for_Wordcloud(neg_words[0:15])
neu_worldCloud=text_for_Wordcloud(neu_words[0:15])
pos_worldCloud=text_for_Wordcloud(pos_words[0:15])

In [44]:
neg_worldCloud

u''

In [45]:
neu_worldCloud

u' maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria maria  donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion donacion  mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila mochila  rico rico rico  puerto puerto puerto  years years  suffering suffering  gloria gloria  plaza plaza  fondo fondo  liz liz  mujeres mujeres  irma irma  donation donation  mejia '

In [46]:
pos_worldCloud

u' work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work work wo